# Kubernetes

In [1]:
#!pip install ipynb
from ipynb.fs.full.facturxWidget import *

from ipynb.fs.full.kubernetesTool import *

from ipynb.fs.full.invoiceSmartcontract import *

from ipynb.fs.full.blockchain import *

In [2]:
_blockchain = Blockchain(channel_name='invoicing27',shared_fs='10.15.130.111:80',sharepoint=SharePoint())
cc = Chaincode()

In [ ]:
cc.__dict__

In [3]:
_blockchain.addPeer(name='aubonmoulin', hostname='fr',ca_country='FR',ca_province='Provence-Alpes-Cote d\'Azur',ca_locality='Malaucene')
_blockchain.addPeer(name='lyon-impots', hostname='gouv.fr',ca_country='FR',ca_province='Auvergne-Rhone-Alpes',ca_locality='Lyon')
_blockchain.addPeer(name='austriafactor', hostname='at',ca_country='AT',ca_province='Steiermark',ca_locality='Graz')
_blockchain.addPeer(name='megustaolive', hostname='es',ca_country='ES',ca_province='Andalusia',ca_locality='Dos Hermanas')

In [ ]:
_blockchain.addPeer(name='aubonmoulin', hostname='fr',ca_country='FR',ca_province='Provence-Alpes-Cote d\'Azur',ca_locality='Malaucene')

In [ ]:
def myfunc(**kwargs):
        # kwargs is a dictionary.
        for k,v in kwargs.items():
             print ("%s = %s" % (k, v))
        #if 'image' in kwargs.keys():
        #    print('found')  
        
        
myfunc(name='aubonmoulin', hostname='fr',ca_country='FR',ca_province='Provence-Alpes-Cote d\'Azur',ca_locality='Malaucene')        

### user

In [ ]:
_blockchain.installWalletTool()

In [ ]:
_user = User(peer=_blockchain.peers['aubonmoulin'], name='Tony.Dubois@aubonmoulin.fr', channel_name=_blockchain.channel_name)
#User(peer=_blockchain.peers['aubonmoulin'], name='Alexandre.Payet@majolieboutique.net', channel_name=_blockchain.channel_name)

In [ ]:
_user.smart_contract.getAllRoles()

In [ ]:
_keys = _user.smart_contract.getAllKeys()[1]

In [ ]:
_user.smart_contract.deleteAsset(asset=_keys[7])

In [ ]:
for key,_peer in _blockchain.peers.items():
    _organizations = _peer.wallet_tool.getPeerOrganizations()
    for _organization in _organizations:
        porg = Organization(name=_organization)
        _users = _peer.wallet_tool.getPeerUsers(_organization)
        for _user in _users:
            puser = User(peer=_peer, name=_user, channel_name=_blockchain.channel_name)
            porg.addUser(user=puser)
        _peer.organizations[_organization]=porg
    

In [ ]:
_blockchain.peers['aubonmoulin'].organizations['aubonmoulin.fr'].users['Tony.Dubois@aubonmoulin.fr'].__dict__

#_blockchain.peers['lyon-impots'].organizations
#_blockchain.peers

In [ ]:
_smart_contract_aubonmoulin = InvoiceSmartContract(peer=_blockchain.peers['aubonmoulin'], user='Tony.Dubois@aubonmoulin.fr', channel_name=_blockchain.channel_name)

In [ ]:
_smart_contract_lyon_impots = InvoiceSmartContract(peer=_blockchain.peers['lyon-impots'], user='Auguste.Boucher@lyon-impots.gouv.fr', channel_name=_blockchain.channel_name)

In [ ]:
_smart_contract.getSubmittingClientIdentity()
_smart_contract.getAllRoles()

In [ ]:
_blockchain.peers['aubonmoulin'].wallet_tool.addUser('Tony.Dubois','aubonmoulin.fr','aubonmoulin.fr')
_blockchain.peers['aubonmoulin'].wallet_tool.addUser('Stephanie.Hoarau','hotelsaintdenis.re','aubonmoulin.fr')
_blockchain.peers['aubonmoulin'].wallet_tool.addUser('Alexandre.Payet','majolieboutique.net','aubonmoulin.fr')
_blockchain.peers['austriafactor'].wallet_tool.addUser('Emma.Steiner','austriafactor.at','austriafactor.at')
_blockchain.peers['megustaolive'].wallet_tool.addUser('Pedro.Sanchez','megustaolive.es','megustaolive.es')
_blockchain.peers['lyon-impots'].wallet_tool.addUser('Auguste.Boucher','lyon-impots.gouv.fr','lyon-impots.gouv.fr')

In [ ]:
def appendRoles(peers):
    peer=peers['lyon-impots']
    appendRole(peer,user='Auguste.Boucher@lyon-impots.gouv.fr',role='TaxInspector')
    peer_taxinspector=peer
    #taxinspector=get_person_names_with_role('TaxInspector',roles)[0].split(',')[0].split('=')[1]  
    taxinspector='Auguste.Boucher@lyon-impots.gouv.fr'

    identity_taxinspector = getSubmittingClientIdentity(peer_taxinspector, user=taxinspector)[0]

    peer=peers['aubonmoulin']
    appendRole(peer,user='Tony.Dubois@aubonmoulin.fr',role='Seller',taxInspector=identity_taxinspector)
    #appendRole(peer,user='Stephanie.Hoarau',role='Buyer')
    #appendRole(peer,user='Alexandre.Payet',role='Buyer')

    peer=peers['austriafactor']
    appendRole(peer,user='Emma.Steiner@austriafactor.at',role='Factor',taxInspector=identity_taxinspector)

    peer=peers['megustaolive']
    appendRole(peer,user='Pedro.Sanchez@megustaolive.es',role='Buyer',taxInspector=identity_taxinspector)
    appendRole(peer,user='Pedro.Sanchez@megustaolive.es',role='Seller',taxInspector=identity_taxinspector)

In [ ]:
def getUser(_blockchain, _user):
    _org = _user.split('@')[1]
    for key,peer in _blockchain.peers.items():
        if _org in peer.organizations.keys():
            if _user in peer.organizations[_org].users.keys():
                return peer.organizations[_org].users[_user]

In [ ]:
getUser(_blockchain,'Auguste.Boucher@lyon-impots.gouv.fr').smart_contract.appendRole(role='TaxInspector')
identity_taxinspector = getUser(_blockchain, 'Auguste.Boucher@lyon-impots.gouv.fr').smart_contract.getSubmittingClientIdentity()[0]
getUser(_blockchain,'Emma.Steiner@austriafactor.at').smart_contract.appendRole(role='Factor',taxInspector=identity_taxinspector)
getUser(_blockchain,'Tony.Dubois@aubonmoulin.fr').smart_contract.appendRole(role='Seller',taxInspector=identity_taxinspector)
getUser(_blockchain,'Pedro.Sanchez@megustaolive.es').smart_contract.appendRole(role='Seller',taxInspector=identity_taxinspector)
getUser(_blockchain,'Pedro.Sanchez@megustaolive.es').smart_contract.appendRole(role='Buyer',taxInspector=identity_taxinspector)

In [ ]:
_keys = getUser(_blockchain,'Tony.Dubois@aubonmoulin.fr').smart_contract.getAllKeys()
_keys

In [ ]:
_smart_contract = getUser(_blockchain,'Tony.Dubois@aubonmoulin.fr').smart_contract

In [ ]:
_keys = getUser(_blockchain,'Tony.Dubois@aubonmoulin.fr').smart_contract.getAllKeys()[0]
_smart_contract = getUser(_blockchain,'Tony.Dubois@aubonmoulin.fr').smart_contract
_smart_contract.deleteAsset(asset=_keys[4])
_smart_contract.deleteAsset(asset=_keys[5])
_smart_contract.deleteAsset(asset=_keys[6])
_smart_contract.deleteAsset(asset=_keys[7])

In [ ]:
_smart_contract.getAllRoles()

In [ ]:
getUser(_blockchain,'Tony.Dubois@aubonmoulin.fr').smart_contract.getRoles()

In [ ]:
#_smart_contract_lyon_impots.getAllRoles()
_smart_contract_aubonmoulin.getRoles()
#_out
_smart_contract_lyon_impots.getRoles()
_smart_contract_lyon_impots.getAllKeys()

In [ ]:
_smart_contract_lyon_impots.getAllKeys()

## cleanup

In [ ]:
_blockchain.deletePeers()
#_blockchain.peers

## add channel

In [ ]:
_blockchain.add_channel('invoicing27')

## debug

In [ ]:
out2.outputs[1]['text'].splitlines()

In [ ]:
_out.outputs[0]['text'].splitlines()

## create peers

### endorse and join

In [ ]:
## endorse and join
import ipywidgets as widgets

#orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host

In [4]:
_blockchain.endorse_and_join_all_peers()

Output()

## upload chaincode to sharepoint

In [ ]:
strlist = ['/bin/sh /opt/create-archive.sh',cc.hostname,cc.port,cc.label,cc.shared_fs]
cmd = toCmd(strlist)
print(cmd)
#cmd = export_cmd + '  && ' + cmd
output = executePeerCmd(getPod('cc-deployer'), cmd,cli='cc-deployer')

## install chaincode

In [5]:
cc=Chaincode(hostname='otest.example.com',service_name='otest')

In [141]:
#cc.__dict__
sc= BasicSmartContract()
callopt= '{"function":"BalanceOf","Args":[' + '\"' + 'asset' + '\"' + ']}'
#callopt= '{"function":"Name","Args":[' + '\"' + 'asset' + '\"' + ']}'
#callopt= '{"function":"Init","Args":[' + '\"' + 'asset' + '\"' + ']}'
callopt
#sc._query(callopt)
#query(self,callopt, channel_name=None, user='User1')
_blockchain.peers['aubonmoulin'].query(callopt, channel_name=_blockchain.channel_name, user='User1')[0]

0

In [147]:
class GolangFunction(BaseAction):
    def __init__(self,contract_name=None,func_name=None,interface=None,args=None,results=None):
        super(GolangFunction, self).__init__()
        self.contract_name=contract_name
        self.func_name=func_name
        self.interface=interface
        self.args=args
        self.results=results
        
class GolangSmartcontract(BaseAction):
    def __init__(self,filename=None):
        super(GolangSmartcontract, self).__init__()
        self.fuctions = {}
        self. init_by_definition_file(filename)
        
    def init_by_definition_file(self,filename):        
       
        with open(filename, 'r') as fp:
            lines = fp.readlines()

        for line in lines:
            _splits = line.split('*')[1].split(')')
            contract_name = _splits[0].strip()
            _ret = line.split(')')[2].strip().strip("(){}").split(',')
            results=[]
            for i in _ret:
                j = i.replace(' ','')
                results.append(j)

            _func_def_split = _splits[1].split('(')
            func_name = _func_def_split[0].strip()
            _arg_split = _func_def_split[1].split(',')
            #print(func_name, _arg_split)
            interface = _arg_split[0]
            
            args = {} 
            for index in range(1,len(_arg_split)):
                arg = _arg_split[index].split()
                args[arg[0]] = arg[1]
            #print(line)    
            #print(func_name,interface,args,result)    
            self.fuctions[func_name] = GolangFunction(contract_name=contract_name,func_name=func_name,interface=interface,args=args,results=results)

In [ ]:
owner = "x509::CN=minter,OU=client,O=Hyperledger,ST=North Carolina,C=US::CN=ca.org1.example.com,O=org1.example.com,L=Durham,ST=North Carolina,C=US"
operator = "x509::CN=org,OU=client,O=Hyperledger,ST=North Carolina,C=US::CN=ca.org1.example.com,O=org1.example.com,L=Durham,ST=North Carolina,C=AR"

In [148]:
_golangSmartcontract = GolangSmartcontract(filename='erc721-contract.go.txt')

In [151]:
#_golangSmartcontract.fuctions.keys()
_golangSmartcontract.fuctions['MintWithTokenURI'].args
_

{'tokenId': 'string', 'tokenURI': 'string'}

'["string","string"]'

In [164]:
def _callopt(function_name=None,args=''):
    
    return '{"function":' + '\"' + function_name + '\"' + ',"Args":' + args + '}'
    
asset = _golangSmartcontract.fuctions['MintWithTokenURI'].args
BasicSmartContract().getArgList(asset)    
callopt = _callopt(function_name='MintWithTokenURI',args=BasicSmartContract().getArgList(asset))    
_blockchain.peers['aubonmoulin'].query(callopt, channel_name=_blockchain.channel_name, user='User1')[1]

Output()

In [172]:
#_blockchain.peers['aubonmoulin'].installWalletTool(sharepoint=_blockchain.sharepoint,channel_name=_blockchain.channel_name)
_blockchain.peers['aubonmoulin'].wallet_tool.getPeerUsers('org1')

ls: /mnt/conf/organizations/peerOrganizations/org1/users/: No such file or directory
command terminated with exit code 1


["Command '['/home/jovyan/work/usr/bin/kubectl', 'exec', '-t', 'aubonmoulin-765bb54cf6-v2q6l', '-c', 'scray-peer-cli', '--', '/bin/sh', '-c', '/bin/ls /mnt/conf/organizations/peerOrganizations/org1/users/']' returned non-zero exit status 1."]

In [6]:
#_blockchain.__dict__
_blockchain.peers['aubonmoulin'].installAndAproveChaincode(orderer=_blockchain.orderer, channel_name=_blockchain.channel_name, chain_code=cc)  


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   424  100   424    0     0  53000      0 --:--:-- --:--:-- --:--:-- 60571
2022-04-12 15:03:38.544 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 001 Installed remotely: response:<status:200 payload:"\nJbasic_1.0:42353bbf8d056d100fd0460fb93a4e4b6c5973aee190345bcdf28abf43fb4754\022\tbasic_1.0" > 
2022-04-12 15:03:38.544 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 002 Chaincode code package identifier: basic_1.0:42353bbf8d056d100fd0460fb93a4e4b6c5973aee190345bcdf28abf43fb4754
Error: query failed with status: 404 - namespace basic is not defined
2022-04-12 15:03:40.862 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [e1cfb1b054ab72c30d996e81fa779a70e1a9bbd38d0b0af98fcab4f84ce37272] committed with status (VALID) at peer0.aubonmoulin.fr:32432


"OK: 42 MiB in 45 packages\nInstalled chaincodes on peer:\nPackage ID: basic_1.0:42353bbf8d056d100fd0460fb93a4e4b6c5973aee190345bcdf28abf43fb4754, Label: basic_1.0\nChaincode definition for chaincode 'basic', version '1.0', sequence '1' on channel 'invoicing27' approval status by org:\nAdminOrgMSP: false\naubonmoulinMSP: true\naustriafactorMSP: false\nlyon-impotsMSP: false\nmegustaoliveMSP: false\n"

In [7]:
_blockchain.installAndAproveChaincode( chain_code=cc)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   424  100   424    0     0  60571      0 --:--:-- --:--:-- --:--:-- 60571
Error: chaincode install failed with status: 500 - failed to invoke backing implementation of 'InstallChaincode': chaincode already successfully installed
Error: query failed with status: 404 - namespace basic is not defined
Error: proposal failed with status: 500 - failed to invoke backing implementation of 'ApproveChaincodeDefinitionForMyOrg': attempted to redefine uncommitted sequence (1) for namespace basic with unchanged content
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   424  100   424    0     0  53000      0 --:--:-- --:--:-- --:--:-- 53000
2022-04-12 15:04:03.177 UTC [cli.lifecycle.chaincode] submitInstallProposal -> INFO 001 

In [8]:
_blockchain.peers['aubonmoulin'].commitChaincode(_blockchain.channel_name,pkgid=cc.pkgid)

/mnt/conf/peer/cc_commit.sh invoicing27 1 basic_1.0:42353bbf8d056d100fd0460fb93a4e4b6c5973aee190345bcdf28abf43fb4754


2022-04-12 15:04:23.617 UTC [chaincodeCmd] ClientWait -> INFO 001 txid [37cb98f386263174af00551b96f9c02c11cfc988105ba2ca8920a8f0ab4e7f8e] committed with status (VALID) at peer0.aubonmoulin.fr:32432


'Installed chaincode\nInstalled chaincodes on peer:\nPackage ID: basic_1.0:42353bbf8d056d100fd0460fb93a4e4b6c5973aee190345bcdf28abf43fb4754, Label: basic_1.0\n'

## add roles

In [ ]:
deleteAllInvoices(User(peer=_blockchain.peers['aubonmoulin'], name='Tony.Dubois@aubonmoulin.fr', channel_name=_blockchain.channel_name).smart_contract)              

In [ ]:
createInvoices(invoices,_blockchain)            

In [ ]:
from typing import Dict, Any
import hashlib
import json

def dict_hash(dictionary: Dict[str, Any]) -> str:
    """MD5 hash of a dictionary."""
    dhash = hashlib.md5()
    # We need to sort arguments so {'a': 1, 'b': 2} is
    # the same as {'b': 2, 'a': 1}
    encoded = json.dumps(dictionary, sort_keys=True).encode()
    dhash.update(encoded)
    return dhash.hexdigest()


In [ ]:
getUser(_blockchain,get_CN_of_invoice(invoices.iloc[3],'sellerTradeParty')).__dict__

In [ ]:
invoice = invoices.iloc[3]
seller = getUser(_blockchain,get_CN_of_invoice(invoice,'sellerTradeParty'))
buyer  = getUser(_blockchain,get_CN_of_invoice(invoice,'buyerTradeParty')) 
seller_id = seller.smart_contract.getSubmittingClientIdentity()[0]
buyer_id  = buyer.smart_contract.getSubmittingClientIdentity()[0]
asset     = seller.smart_contract.getEmptyInvoice()[0]
update_invoice(asset=asset,invoice=invoice,id=3,seller=seller_id,buyer=buyer_id)
seller.smart_contract.createInvoice(arglist=getArgList(asset))[1]

In [ ]:
seller.smart_contract.getAllKeys()

In [ ]:
seller.smart_contract.listInvoice(id='asset14')

In [ ]:
seller_id

In [ ]:
User(peer=_blockchain.peers['aubonmoulin'], name='Tony.Dubois@aubonmoulin.fr', channel_name=_blockchain.channel_name).smart_contract.listInvoices()

In [ ]:
invoice=invoices.iloc[3]
getUser(_blockchain,get_CN_of_invoice(invoice,'sellerTradeParty'))

In [ ]:
seller.smart_contract.getAllKeys()[0]

In [ ]:
invoices = get_invoices()
#createInvoices(invoices,_blockchain.peers)
#invoices

In [ ]:
def get_org_of_invoice(invoice,party):
    _name=strip_accents(getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/' + party + '/definedTradeContact/')[0]['personName']['value'].replace(' ','.'))
    _email=getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/' + party + '/definedTradeContact/')[0]['emailURIUniversalCommunication']['uriid']['value']
    return _email.split('@')[1]

In [ ]:
User(peer=_blockchain.peers['aubonmoulin'], user='Tony.Dubois@aubonmoulin.fr', channel_name=_blockchain.channel_name).smart_contract.getSubmittingClientIdentity()

In [ ]:
get_CN_of_invoice(invoice,'sellerTradeParty')

In [ ]:
#list(asset)
_smart_contract_aubonmoulin.getEmptyInvoice()[0]

In [ ]:
InvoiceSmartContract(peer=_blockchain.peers['lyon-impots'], user='Auguste.Boucher@lyon-impots.gouv.fr').appendRole(role='TaxInspector')

In [ ]:
InvoiceSmartContract(peer=_blockchain.peers['aubonmoulin'], user='Tony.Dubois@aubonmoulin.fr').appendRole(role='Seller', taxInspector='')

In [ ]:
strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/']
peer=_blockchain.peers['aubonmoulin']
executePeerCmd(peer.pod_name,toCmd(strlist)).splitlines() 

In [ ]:
appendRoles(_blockchain.peers)
#peer=_blockchain.peers['lyon-impots']
#appendRole(peer,user='Auguste.Boucher@lyon-impots.gouv.fr',role='TaxInspector')

In [ ]:
peer=_blockchain.peers['lyon-impots']
getRoles(peer,user='Auguste.Boucher@lyon-impots.gouv.fr')[0]

In [ ]:
callopt='{"function":"AppendRole","Args":["x509::CN=Auguste.Boucher@lyon-impots.gouv.fr,OU=admin,O=kubernetes.research.dev.seeburger.de::CN=ca.lyon-impots.gouv.fr,O=lyon-impots.gouv.fr,L=Lyon,ST=Auvergne-Rhone-Alpes,C=FR","TaxInspector",""]}'

In [ ]:
peer=_blockchain.peers['lyon-impots']
getAllKeys(peer)[1]

In [ ]:
peer=_blockchain.peers['lyon-impots']
callopt= '{"function":"GetAllKeys","Args":[' + ']}'
#peer.query(callopt,channel_name='invoicing18',user='Auguste.Boucher@lyon-impots.gouv.fr')
peer.query(callopt,channel_name='invoicing18',user='User1')

In [ ]:
getAllRoles(peer)
getRoles(peer, user='Auguste.Boucher@lyon-impots.gouv.fr')

In [ ]:
getRoleTransactions(peer)

In [ ]:
invoices = get_invoices()
createInvoices(invoices,_blockchain.peers)
#invoices

In [ ]:
peer=_blockchain.peers['aubonmoulin']
getPeerOrganizations(peer)

In [ ]:
get_peer_of_user('Tony.Dubois@aubonmoulin.fr',_blockchain.peers)

In [ ]:
peer=_blockchain.peers['aubonmoulin']
listInvoices(peer, user='Tony.Dubois@aubonmoulin.fr')[1]

In [ ]:
peer=_blockchain.peers['aubonmoulin']
InvoiceSmartContract(peer=peer, user='Tony.Dubois@aubonmoulin.fr').listInvoices()[0]

In [ ]:
InvoiceSmartContract(peer=peer, user='Tony.Dubois@aubonmoulin.fr').__dict__

In [ ]:
deleteAllInvoices(peer)  